# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat publications.tsv

SyntaxError: invalid syntax (<ipython-input-2-ebaab4945948>, line 1)

## Import pandas

We are using the very handy pandas library for dataframes.

In [1]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [33]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,1/1/2019,Roles of irrigation and reservoir operations i...,Journal of Geophysical Research: Atmosphere,This paper examines water management impact ov...,NaN,JGR-Indian-VIC-IRR-Reservoir,https://doi.org/10.1029/2019MS001870
1,1/2/2019,The DOE E3SM coupled model version 1: Descript...,Journal of Advances in Modeling Earth Systems,This paper overviews E3SM V1 at high resolutio...,NaN,JAMES-E3SM-overview-high-resolution,https://doi.org/10.1029/2019MS001870
2,1/3/2019,Flood Inundation Generation Mechanisms and The...,Journal of Geophysics Research: Atmospheres,This paper used MOSART-inundation to investiga...,NaN,JGR-Flood-Inundation,https://doi.org/10.1029/2019JD031381
3,1/4/2019,The DOE E3SM Coupled Model Version 1: Overview...,Journal of Advances in Modeling Earth Systems,This paper overviews E3SM V1 at standard resol...,"Golaz, J. C., Caldwell, P. M., Van Roekel, L. ...",JAMES-E3SM-overview,https://doi.org/10.1029/2018MS001603
4,1/5/2019,Strong influence of irrigation on water budget...,Journal of Geophysical Research: Atmospheres,This paper examines irrigation impacts over In...,NaN,JGR-Indian-VIC-IRR,https://doi.org/10.1029/2018JD029132
5,1/1/2018,Modulating factors of hydrologic exchanges in ...,Hydrological Processes,This paper investigates the major control fact...,NaN,HP-Columbia-River-CFD,https://doi.org/10.1002/hyp.13266
6,1/2/2018,Non-stationary hydropower generation projectio...,Environmental Research Letters,This paper compares process-based model predic...,NaN,ERL-Hydropower,https://doi.org/10.1088/1748-9326/aad19f
7,1/3/2018,Evaluating the functionality and streamflow im...,Hydrological Processes,This paper examines the effect of forest gaps ...,NaN,HP-Snow-Gap,https://doi.org/10.1002/hyp.13150
8,1/4/2018,Riverbed hydrologic exchange dynamics in a lar...,Water Resources Research,This paper examines how upstream dam operation...,NaN,WRR-Columbia-River-CFD,https://doi.org/10.1002/2017WR020508
9,1/5/2018,Sensitivity of regulated flow regime to climat...,Journal of Hydrometeorology,This paper compares the regulated flow and nat...,NaN,JHM-West-US-WM,https://doi.org/10.1175/JHM-D-17-0095.1


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [34]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [35]:
import os
for row, item in publications.iterrows():
    
    year = item.pub_date[4:8]
    md_filename = str(year) + "-" + item.url_slug + ".md"
    html_filename = str(year) + "-" + item.url_slug
        
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    # md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Link to the paper](" + item.paper_url + ")\n" 
    
    md += "\n![image](../images/papers/" + str(year) + "-" + item.url_slug + ".png)\n"
    
    # md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)
        
    item.pub_date

In [24]:
item.pub_date[4:8]

'2012'

These files are in the publications directory, one directory below where we're working from.

In [11]:
!ls ../_publications/

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---
This paper is about the number 1. The number 2 is left for future work.

[Download paper here](http://academicpages.github.io/files/paper1.pdf)

Recommended citation: Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).